In [2]:
import numpy as np
import pandas as pd
from pykalman import KalmanFilter
from fancyimpute import IterativeImputer

import numpy as np
import pandas as pd
import os

import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pyampute
import pickle 
import time

from scipy.stats import mstats
from scipy.stats.mstats import winsorize
from scipy import stats
from xgboost import XGBRegressor
from sklearn import tree
from pyampute.ampute import MultivariateAmputation
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from lifelines import CoxPHFitter, WeibullFitter, WeibullAFTFitter
from datetime import datetime, date, timedelta
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from os.path import isfile, join
from sklearn.metrics import mean_absolute_error, roc_auc_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from statsmodels.gam.tests.test_penalized import df_autos
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import norm
from scipy.spatial import distance
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import warnings 
warnings.filterwarnings('ignore')

from pyampute.ampute import MultivariateAmputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from data import load_dataset 
from app_hyperparameters import init_parameters_decision_tree, init_parameters_xgboost 
from app_hyperparameters import init_parameters_bayesian_ridge, init_parameters_random_forest
from app_stopping_criteria import stop_iteration
from app_uncertainty import uncertainty_sampling, multi_argmax, imputation_uncertainty
from app_uncertainty import EI
from app_init import init_truncation, init_variable_schema
from app_init import init_truncation, init_variable_schema, init_imputation_columns

import miceforest as mf 
import random

drive = 'H'
main_path = drive + ':/Shared drives/CKD_Progression/data/CKD_COHORT_Jan2010_Mar2024_v3.csv'
data_path = drive + ':/Shared drives/CKD_Progression/data/'
docs_path = drive + ':/Shared drives/CKD_Progression/docs/'
save_path = drive + ':/Shared drives/CKD_Progression/save/'
resu_path = drive + ':/Shared drives/CKD_Progression/result/'
covariates_path = docs_path + 'covariates.csv'
removecols_path = docs_path + 'remove_columns.csv'

In [3]:
def convert_to_int_list(input_str):
    result_list = []
    for i, char in enumerate(input_str):
        if char == '0':
            continue
        else:
            result_list.append(f'{i} ')
    return result_list

def generate_miss_pattern(df, columns, mechanism = 'MAR', threshold=False, top = False):
    '''
    Parameters:
        df (dataframe): original dataframe  (with missing values)
        columns (list): columns with missing values
        mechanism (string): MAR (default)
                            MNAR: missing not at random
        top (int): top n patterns 
    Returns result (list): a list of dictionaries with its corresponding missingness pattern and frequency
    '''
    miss_pattern = df[columns]
    miss_pattern = miss_pattern.isna().astype(int)
    miss_pattern.columns = np.arange(1, len(miss_pattern.columns) + 1, 1).tolist()

    miss_pattern['combined'] = miss_pattern.apply(lambda row: ''.join(map(str, row)), axis = 1)
    miss_pattern['combined'] = miss_pattern['combined'].apply(convert_to_int_list)
    "if there are complete rows, don't count them"
    miss_pattern = miss_pattern[miss_pattern["combined"].apply(lambda x: len(x)>0)]
    
    miss_pattern = pd.DataFrame(miss_pattern['combined'].value_counts(ascending = False))
    if top is not False:
        miss_pattern = miss_pattern.head(top)
    miss_pattern['freq'] = miss_pattern['combined']/miss_pattern['combined'].sum()
    "if we use threshold, remove rows lower than that, then recalculate frequencies"
    if threshold is not False:
        miss_pattern = miss_pattern[miss_pattern['freq']>=threshold]
        miss_pattern['freq'] = miss_pattern['combined']/miss_pattern['combined'].sum()
    miss_pattern = miss_pattern.reset_index().rename(columns = {'index': 'incomplete_vars'})
    miss_pattern = miss_pattern.drop('combined', axis = 1)
    miss_pattern['mechanism'] = mechanism

    result = []
    for index, row in miss_pattern.iterrows():
        incomplete_vars = row['incomplete_vars']
        result.append({
            'incomplete_vars': list(set(map(int, row['incomplete_vars']))),
            'mechanism': row['mechanism'],
            'freq': row['freq']})
    return result

In [4]:
def generate_miss_pattern(df, columns, mechanism='MAR', threshold=False, top=False):
    miss_pattern = df[columns].isna().to_numpy().astype(int)
    combined = np.dot(miss_pattern, 1 << np.arange(miss_pattern.shape[1] - 1, -1, -1))

    unique, counts = np.unique(combined, return_counts=True)
    freq = counts / counts.sum()

    if threshold:
        mask = freq >= threshold
        unique, counts, freq = unique[mask], counts[mask], freq[mask]

    if top:
        top_indices = np.argsort(counts)[-top:][::-1]
        unique, freq = unique[top_indices], freq[top_indices]

    result = [{'incomplete_vars': [i + 1 for i in range(len(columns)) if (u >> i) & 1],
               'mechanism': mechanism,
               'freq': f} for u, f in zip(unique, freq)]
    return result

def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return filtered_df

In [4]:
covariates, order_covariates, long_df = load_dataset(get_columns = True)
patients = long_df['ENC_HN'].unique().tolist()

In [24]:
columns_impute = init_imputation_columns()
df = pd.read_csv(docs_path + 'CKD_TVC_IMPUTATION_14December2024.csv')
df = df.rename(columns = {'height': 'HIGH'})
df = df[['ENC_HN', 'modulo_365'] + columns_impute]
df = remove_outliers(df, 'BMI')
df = df[df['modulo_365'] != 14]

In [6]:
df.describe(percentiles = [0.01, 0.10, 0.25, 0.50, 0.75, 0.99]).T.to_csv(docs_path + 'descriptive_longitudinal.csv')

In [28]:
missing_rate = df.groupby('ENC_HN').apply(
    lambda group: group.isna().mean())
missing_rate = missing_rate.drop(columns = ['modulo_365', 'ENC_HN']).reset_index(drop = True)
missing_rate.describe().T.to_csv(docs_path + 'missing_rate_longitudinal_2.csv')

In [6]:
def generate_miss_pattern(df, columns, mechanism = 'MAR', threshold = False, top = False):
    miss_pattern = df[columns].isna().to_numpy(dtype=np.uint8)
    powers = 1 << np.arange(miss_pattern.shape[1] - 1, -1, -1, dtype=np.int64)
    combined = miss_pattern.dot(powers).astype(np.int64)

    unique, counts = np.unique(combined, return_counts=True)
    total_count = counts.sum()
    freq = counts / total_count

    if threshold:
        mask = freq >= threshold
        unique, freq = unique[mask], freq[mask]

    if top:
        top_indices  = np.argpartition(counts, -top)[-top:]
        unique, freq = unique[top_indices], freq[top_indices]
        sorted_order = np.argsort(-counts[top_indices])
        unique, freq = unique[sorted_order], freq[sorted_order]

    result = [
        {
            'incomplete_vars': [i + 1 for i in range(len(columns)) if (u >> i) & 1],
            'mechanism': mechanism,
            'freq': f
        }
        for u, f in zip(unique, freq)]
    return result

In [22]:
# patient_missing_data, missing_data_desc = get_patient_missing_data(df)

patient_missing_data = pd.read_csv(docs_path + 'missing_rate_longitudinal.csv')
patient_missing_data = patient_missing_data[patient_missing_data['ENC_HN'].isin(patients)].reset_index(drop = True)
missing_data_desc = patient_missing_data.describe().T.reset_index()
missing_data_desc

,index,count,mean,std,min,25%,50%,75%,max
0,BMI,23429.0,0.097054,0.268651,0.0,0.000000,0.000000,0.000000,1.0
1,BW,23429.0,0.074169,0.238300,0.0,0.000000,0.000000,0.000000,1.0
2,age,23429.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,DBP,23429.0,0.181430,0.264808,0.0,0.000000,0.083333,0.250000,1.0
4,SBP,23429.0,0.181596,0.264728,0.0,0.000000,0.083333,0.250000,1.0
5,Lipid_Cholesterol,23429.0,0.354286,0.319069,0.0,0.076923,0.285714,0.545455,1.0
6,Lipid_LDL,23429.0,0.445036,0.356468,0.0,0.142857,0.333333,0.750000,1.0
7,Lipid_Triglyceride,23429.0,0.468600,0.340320,0.0,0.181818,0.416667,0.750000,1.0
8,Chem_HbA1C,23429.0,0.577346,0.370588,0.0,0.230769,0.615385,1.000000,1.0


In [6]:
columns_impute   = ['BMI', 'BW', 'age', 'DBP', 'SBP', 'Lipid_Cholesterol', 'Lipid_LDL','Lipid_Triglyceride', 'Chem_HbA1C']
missing_patterns = {} 
proportion_denom = {} 
for patient in tqdm(df['ENC_HN'].unique().tolist()):
    patient_df = df[df['ENC_HN'] == patient]
    missing_pattern = generate_miss_pattern(patient_df, columns_impute, threshold = 0.01, top = False)
    missing_patterns[patient] = missing_pattern
    proportion_den = len(patient_df)
    proportion_denom[patient] = proportion_den

In [9]:
# complete_df = df.dropna(subset = [col for col in df.columns if col not in columns_low_missing])
complete_df = df.dropna().reset_index(drop = True)
complete_df['flag'] = 1
complete_df['count'] = complete_df.groupby('ENC_HN')['flag'].transform('sum')

complete_df = complete_df[complete_df['count'] >= 5].reset_index(drop = True)
complete_df = complete_df.drop(columns = ['flag', 'count'])

In [10]:
complete_df.isnull().sum()/complete_df.shape[0]

ENC_HN                0.0
modulo_365            0.0
BMI                   0.0
BW                    0.0
age                   0.0
DBP                   0.0
SBP                   0.0
Lipid_Cholesterol     0.0
Lipid_LDL             0.0
Lipid_Triglyceride    0.0
Chem_HbA1C            0.0
dtype: float64

In [12]:
import warnings 
warnings.filterwarnings('ignore')

def ampute_dataset(da, missing_pattern, columns_ampute):
    seed = 100
    df_list = [] 

    for patient in da['ENC_HN'].unique().tolist():
        complete = da[da['ENC_HN'] == patient][columns_ampute]

        pattern = missing_pattern.get(patient)
        pattern = [d for d in pattern if d['incomplete_vars']]
        if not pattern:
            continue

        total_freq = sum(d['freq'] for d in pattern)
        if total_freq != 1:
            remaining_diff = 1 - total_freq
            pattern[-1]['freq'] += remaining_diff 

        len_patient_data = proportion_denom[patient]
        proportion = (len_patient_data - len(complete)) / len_patient_data
        amputer = MultivariateAmputation(prop = proportion, seed = seed, patterns = pattern)
        incomplete = amputer.fit_transform(complete)
        incomplete_df = pd.DataFrame(incomplete, columns = columns_ampute)
        incomplete_df['ENC_HN'] = patient
        df_list.append(incomplete_df)
    incomplete_data = pd.concat(df_list, axis = 0)
    return incomplete_data, amputer

columns_ampute   = ['BMI', 'BW', 'age', 'DBP', 'SBP', 'Lipid_Cholesterol', 'Lipid_LDL','Lipid_Triglyceride', 'Chem_HbA1C']
# try:
#     ampute_data, amputer = ampute_dataset(complete_df, missing_patterns, columns_ampute)
# except ValueError as e:
#     print(f"Error: {e}")

incomplete_data, amputer = ampute_dataset(complete_df, missing_patterns, columns_ampute)

In [25]:
def estimate_transition_matrix(data):
    if data.shape[0] < 2:
        return np.eye(1)  
    
    X_t = data[:-1]
    X_t_plus_1 = data[1:]
    A, _, _, _ = np.linalg.lstsq(X_t, X_t_plus_1, rcond=None)
    return A.reshape(1, 1)

def impute(data, n_datasets, variable_schema, truncation, time_variable, 
           cov_threshold = 0.5, id_col = 'ENC_HN', n_iteration = 50, 
           convergence = ['maxit', 'delta', 'early_stop'], model='linear', 
           seed = 1997, verbose = True):
    df_list, delta_change, convergence_data = [], [], []
    for m in range(n_datasets):
        if verbose:
            print(f'Dataset number {m + 1}')

        filled_df = data.copy()
        mask = filled_df.isna()
        imp_order = filled_df[list(variable_schema.keys())].isna().sum().sort_values(ascending=False).index

        for to_impute in imp_order:
            np.random.seed(seed)
            local_mu = filled_df[to_impute].mean()
            local_sd = filled_df[to_impute].std()
            guesses = np.random.normal(loc   = local_mu, 
                                       scale = local_sd,
                                       size  = len(filled_df))
            filled_df.loc[mask[to_impute], to_impute] = guesses[mask[to_impute]]
        it, delta = 0, []
        while it < n_iteration:
            if verbose:
                print(f'Iteration {it + 1}', end = ', ')
            pre_it = filled_df[imp_order].copy()

            for patient_id, patient_data in filled_df.groupby(id_col):
                for to_impute in imp_order:
                    if to_impute in time_variable:
                        covariates = patient_data[variable_schema[to_impute]].values
                        observed_data = patient_data[to_impute].fillna(0).values.reshape(-1, 1)
                        A = estimate_transition_matrix(observed_data)
                        kf = KalmanFilter(transition_matrices = A,
                                          transition_covariance = 0.1 * np.eye(A.shape[0]), observation_covariance = 1.0,
                                          initial_state_mean  = [observed_data.mean()])
                        state_means, state_covariances = kf.smooth(observed_data)

                        reg = LinearRegression()
                        X_train = np.hstack([state_means, covariates])

                        reg.fit(X_train, observed_data.flatten())

                        X_test = np.hstack([state_means, patient_data[variable_schema[to_impute]].values])
                        y_pred = reg.predict(X_test)

                        cov_diag = np.array([np.diag(cov) for cov in state_covariances]).flatten()
                        low_cov_samples = cov_diag < cov_threshold
                        valid_indices = patient_data.index[low_cov_samples]

                        bounds = truncation[to_impute]
                        y_pred = np.clip(y_pred, bounds[0], bounds[1])
                        y_pred = winsorize(y_pred, limits = (0.10, 0.10))

                        filled_df.loc[valid_indices, to_impute] = (
                            filled_df.loc[valid_indices, to_impute]
                            .where(~mask.loc[valid_indices, to_impute], y_pred[low_cov_samples]))
                    else:
                        sampled_data = pre_it[~mask[to_impute]]
                        X_train = sampled_data[variable_schema[to_impute]]
                        y_train = sampled_data[to_impute]

                        estimator = BayesianRidge()
                        estimator.fit(X_train, y_train)

                        y_imputed = estimator.predict(filled_df[variable_schema[to_impute]])

                        bounds = truncation[to_impute]
                        y_imputed = np.clip(y_imputed, bounds[0], bounds[1])
                        y_imputed = winsorize(y_imputed, limits=(0.10, 0.10))

                        filled_df[to_impute] = filled_df[to_impute].where(~mask[to_impute], y_imputed)
            post_it = filled_df[imp_order].copy()
            delta_val = np.sqrt(((post_it - pre_it) ** 2).sum() / len(post_it))
            delta.append(delta_val)

            if np.nanmax(delta_val) < 1e-4:
                break
            it += 1
        df_list.append(filled_df)
        delta_change.append(pd.concat(delta, axis='columns'))
        convergence_data.append(post_it)

    result = {
        'imputed_data': df_list,
        'convergence_data': convergence_data,
        'iteration_delta': delta_change}
    return result

In [26]:
def init_variable_schema():
    variable_schema = {
        'BW': ['Lipid_Cholesterol', 'Lipid_LDL', 'Lipid_Triglyceride'],
        'Lipid_LDL': ['Lipid_Cholesterol', 'Lipid_Triglyceride', 'BW'],
        'Chem_HbA1C': ['Lipid_Cholesterol', 'Lipid_LDL', 'Lipid_Triglyceride'],
        'Lipid_Cholesterol': ['Lipid_LDL', 'Lipid_Triglyceride', 'BW'],
        'Lipid_Triglyceride': ['Lipid_Cholesterol', 'Lipid_LDL', 'BW'], }
    return variable_schema

def pool_imputed_data(imputed_data):
    numeric_columns = imputed_data[0].select_dtypes(include=[np.number]).columns
    stacked_data = np.stack([df[numeric_columns].values for df in imputed_data], axis=0)
    pooled_mean = np.nanmean(stacked_data, axis=0)
    pooled_df = pd.DataFrame(pooled_mean, columns=numeric_columns)
    for col in imputed_data[0].columns:
        if col not in numeric_columns:
            pooled_df[col] = imputed_data[0][col]
    return pooled_df

In [27]:
columns_complete = []
imputed_data = impute(data = df, 
                      n_datasets = 5, 
                      variable_schema = init_variable_schema(), 
                      truncation = init_truncation(), 
                      n_iteration = 10, 
                      convergence = 'early_stop',
                      seed = 1996, 
                      model = 'linear', 
                      time_variable = columns_impute,
                      verbose = True)

imputed_data_list = imputed_data['imputed_data'] 
pooled_result = pool_imputed_data(imputed_data_list)
pooled_result = pooled_result[['ENC_HN','modulo_365', 'BW', 'age', 'Lipid_Cholesterol', 'Lipid_LDL', 'Lipid_Triglyceride', 'Chem_HbA1C']]
complete_dataset = pooled_result.copy()

Dataset number 1
Iteration 1, Iteration 2, Iteration 3, Iteration 4, Iteration 5, Iteration 6, Iteration 7, Iteration 8, Iteration 9, Iteration 10, Dataset number 2
Iteration 1, Iteration 2, Iteration 3, Iteration 4, Iteration 5, Iteration 6, Iteration 7, Iteration 8, Iteration 9, Iteration 10, Dataset number 3
Iteration 1, Iteration 2, Iteration 3, Iteration 4, Iteration 5, Iteration 6, Iteration 7, Iteration 8, Iteration 9, Iteration 10, Dataset number 4
Iteration 1, Iteration 2, Iteration 3, Iteration 4, Iteration 5, Iteration 6, Iteration 7, Iteration 8, Iteration 9, Iteration 10, Dataset number 5
Iteration 1, Iteration 2, Iteration 3, Iteration 4, Iteration 5, Iteration 6, Iteration 7, Iteration 8, Iteration 9, Iteration 10, 

In [ ]:
missing_patterns = {} 
for patient in df['ENC_HN'].unique().tolist():
    patient_df = df[df['ENC_HN'] == patient]
    missing_pattern = generate_miss_pattern(patient_df, columns_impute, threshold = 0.15, top = False)
    missing_patterns[patient] = missing_pattern